In [1]:
import tensorflow
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
%matplotlib inline

In [2]:
def gimme_json(img_directory):
    jpg_img_directory = img_directory + '/'
    img_directory = img_directory + '_binary/' # After this it's like './Bacteria_binary/'
    jpg_file_list = os.listdir(jpg_img_directory)
    file_list = os.listdir(img_directory)

    j = 0
    
    for ( file, jpg_file ) in zip(file_list, jpg_file_list) :
        file_name = file
        file_path = img_directory + file_name # After this it's like ./Bacteria/m0039.png
        jpg_file_path = jpg_img_directory + jpg_file
        
        img = plt.imread(file_path)
        img = img[:, :, 0]
        img = img.astype(np.uint8)
        img_copy = np.copy(img)

        _, thresh = cv2.threshold(img_copy, 0, 1, 0)
        _, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)


        name_string = str(file_name[:-3]) + 'jpg'
        file_size = os.path.getsize(jpg_file_path)
        file_size_string = str(file_size)
        name_and_size_string = '"' + name_string + file_size_string + '"'


        if j > 0:
            main_str = main_str + ','
            main_str = main_str + name_and_size_string + ':' + '{' + '"' + 'filename' + '"' + ':' + '"' + name_string + '"' + ',' + '"' + 'size' + '"' + ':' + file_size_string + ',' + '"' + 'regions' + '"' + ':' + '['
        elif j == 0:
            main_str = '{' + name_and_size_string + ':' + '{' + '"' + 'filename' + '"' + ':' + '"' + name_string + '"' + ',' + '"' + 'size' + '"' + ':' + file_size_string + ',' + '"' + 'regions' + '"' + ':' + '['
            
            
        # main_str = name_and_size_string + ':' + '{' + '"' + 'filename' + '"' + ':' + '"' + name_string + '"' + ',' + '"' + 'size' + '"' + ':' + file_size_string + ',' + '"' + 'regions' + '"' + ':' + '['
        xs = []
        ys = []
        i = 0

        for contour in contours[0:2]:
            all_x_pts = []
            all_y_pts = []
            if i > 0:
                main_str = main_str + ','
            for point in contour:
                x = point[0][0]
                y = point[0][1]
                all_x_pts.append(x)
                all_y_pts.append(y)
            xs.append(all_x_pts)
            ys.append(all_y_pts)

            main_str = main_str + '{' + '"' + 'shape_attributes' + '"' + ':' + '{' + '"' + 'name' + '"' + ':' + '"' + 'polygon' + '"' + ',' + '"' + 'all_points_x' + '"' + ':' + str(all_x_pts) + ',' + '"' + 'all_points_y' + '"' + ':' + str(all_y_pts) + '}' + ',' + '"' + 'region_attributes' + '"' + ':' + '{' + '"' + 'class' + '"' + ':' + '{' + '"' + 'bacteria' + '"' + ':' + 'true' + '}}}'                                                  #
            i = i + 1


        main_str = main_str + ']'

        main_str = main_str + ','  + '"' + 'file_attributes' + '"' + ':{}}'
        
        j = j + 1
        
    main_str = main_str + '}'
    
    json_file = open('./via_region_data.json', 'w')
    json_file.write(main_str)
    json_file.close()

In [3]:
if __name__ == '__main__':
    ROOT_DIR = os.path.abspath("./")
    gimme_json( './datasets/bacteria/train')